[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain-academy/blob/main/module-1/deployment.ipynb) [![Open in LangChain Academy](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66e9eba12c7b7688aa3dbb5e_LCA-badge-green.svg)](https://academy.langchain.com/courses/take/intro-to-langgraph/lessons/58239303-lesson-8-deployment)

# Deployment

## Review 

우리는 메모리를 가진 에이전트를 구축했습니다:

* `act` - 모델이 특정 도구를 호출하도록 합니다
* `observe` - 도구 출력을 모델로 다시 전달합니다
* `reason` - 모델이 도구 출력에 대해 추론하여 다음 행동(예: 다른 도구 호출 또는 직접 응답)을 결정하도록 합니다
* `persist state` - 중단이 발생하는 장기 대화 지원을 위해 메모리 내 체크포인트기 사용

## Goals

이제 에이전트를 로컬 환경의 Studio와 `LangGraph Cloud`에 실제로 배포하는 방법을 살펴보겠습니다.

In [1]:
# %%capture --no-stderr
# %pip install --quiet -U langgraph_sdk langchain_core

## Concepts

`LangGraph` —
- Python 및 JavaScript 라이브러리
- 에이전트 워크플로 생성 지원

`LangGraph API` —
- 그래프 코드 번들링
- 비동기 작업 관리를 위한 작업 대기열 제공
- 상호작용 간 상태 유지 보존 기능 제공

`LangSmith Deployment` (구 `LangGraph Cloud`) --
- LangGraph API 호스팅 서비스
- GitHub 저장소에서 그래프 배포 가능
- 배포된 그래프에 대한 모니터링 및 추적 기능 제공
- 각 배포마다 고유한 URL로 접근 가능

`LangSmith Studio` (구 `LangGraph Studio`) --
- LangGraph 애플리케이션용 통합 개발 환경(IDE)
- API를 백엔드로 사용해 그래프의 실시간 테스트 및 탐색 가능
- 로컬 또는 클라우드 배포로 실행 가능. 아래 참조.

`LangGraph SDK` --
- LangGraph 그래프와 프로그래밍 방식으로 상호작용하기 위한 Python 라이브러리
- 로컬 또는 클라우드에서 제공되는 그래프 작업에 일관된 인터페이스 제공
- 클라이언트 생성, 어시스턴트 접근, 스레드 관리, 실행 수행 가능

## Testing Locally

## Studio

**⚠️ Notice**

이 동영상 촬영 이후, Studio를 업데이트하여 이제 로컬에서 실행하고 브라우저를 통해 접근할 수 있게 되었습니다.   
동영상에 나온 데스크톱 앱을 사용하는 대신 Studio를 실행하는 권장 방법입니다.   
이제 _LangGraph Studio_ 대신 _LangSmith Studio_로 명칭이 변경되었습니다.   
자세한 설정 방법은 과정 시작 부분의 “Getting Setup” 가이드에서 확인할 수 있습니다.   
Studio에 대한 설명은 [여기](https://docs.langchain.com/langsmith/studio)에서, 로컬 배포에 대한 구체적인 내용은 [여기](https://docs.langchain.com/langsmith/quick-start-studio#local-development-server)에서 확인할 수 있습니다.  
로컬 개발 서버를 시작하려면, 이 모듈의 `/studio` 디렉터리에서 터미널에 다음 명령어를 실행하세요:

```
langgraph dev
```

다음과 같은 출력이 표시되어야 합니다:
```
- 🚀 API: http://127.0.0.1:2024
- 🎨 Studio UI: https://smith.langchain.com/studio/?baseUrl=http://127.0.0.1:2024
- 📚 API 문서: http://127.0.0.1:2024/docs
```

브라우저를 열고 위에 표시된 **Studio UI** URL로 이동하세요.

In [2]:
# if 'google.colab' in str(get_ipython()):
#     raise Exception("Unfortunately LangGraph Studio is currently not supported on Google Colab")

In [3]:
from langgraph_sdk import get_client

In [5]:
# This is the URL of the local development server
URL = "http://127.0.0.1:2024"
client = get_client(url=URL)

# Search all hosted graphs
assistants = await client.assistants.search()

In [6]:
assistants

[{'assistant_id': 'fe096781-5601-53d2-b2f6-0d3403f7e9ca',
  'graph_id': 'agent',
  'config': {},
  'context': {},
  'metadata': {'created_by': 'system'},
  'name': 'agent',
  'created_at': '2026-02-19T00:12:40.982565+00:00',
  'updated_at': '2026-02-19T00:12:40.982565+00:00',
  'version': 1,
  'description': None},
 {'assistant_id': '228f9934-0cdd-5383-92c8-ee8422522cc2',
  'graph_id': 'router',
  'config': {},
  'context': {},
  'metadata': {'created_by': 'system'},
  'name': 'router',
  'created_at': '2026-02-19T00:12:40.976770+00:00',
  'updated_at': '2026-02-19T00:12:40.976770+00:00',
  'version': 1,
  'description': None},
 {'assistant_id': '28d99cab-ad6c-5342-aee5-400bd8dc9b8b',
  'graph_id': 'simple_graph',
  'config': {},
  'context': {},
  'metadata': {'created_by': 'system'},
  'name': 'simple_graph',
  'created_at': '2026-02-18T10:43:35.961997+00:00',
  'updated_at': '2026-02-18T10:43:35.961997+00:00',
  'version': 1,
  'description': None}]

In [7]:
assistants[-3]

{'assistant_id': 'fe096781-5601-53d2-b2f6-0d3403f7e9ca',
 'graph_id': 'agent',
 'config': {},
 'context': {},
 'metadata': {'created_by': 'system'},
 'name': 'agent',
 'created_at': '2026-02-19T00:12:40.982565+00:00',
 'updated_at': '2026-02-19T00:12:40.982565+00:00',
 'version': 1,
 'description': None}

In [9]:
# 실행 상태를 추적하기 위한 스레드를 생성합니다
thread = await client.threads.create()

이제 에이전트를 [`client.runs.stream`](https://docs.langchain.com/oss/python/langgraph/graph-api/#stream-and-astream)으로 실행할 수 있습니다:

* `thread_id`
* `graph_id`
* `input`
* `stream_mode`

스트리밍에 대해서는 향후 모듈에서 자세히 다룰 예정입니다.

현재로서는 `stream_mode=“values”`를 사용하여 그래프의 각 단계 이후 상태의 전체 값을 [스트리밍](https://docs.langchain.com/langsmith/streaming)하고 있다는 점만 인지하시면 됩니다.
 
상태는 `chunk.data`에 캡처됩니다. 

In [10]:
from langchain_core.messages import HumanMessage

# Input
input = {"messages": [HumanMessage(content="3과 2를 곱해줘.")]}

# Stream
async for chunk in client.runs.stream(
        thread['thread_id'],
        "agent",
        input=input,
        stream_mode="values",
    ):
    if chunk.data and chunk.event != "metadata":
        print(chunk.data['messages'][-1])

{'content': '3과 2를 곱해줘.', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'human', 'name': None, 'id': 'fda53497-a6eb-4604-92f2-71d36098f866'}
{'content': '', 'additional_kwargs': {'refusal': None}, 'response_metadata': {'token_usage': {'completion_tokens': 17, 'prompt_tokens': 138, 'total_tokens': 155, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_64dfa806c7', 'id': 'chatcmpl-DAm9Y0TTNWoeZglOoAPGdyiaHzAZH', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, 'type': 'ai', 'name': None, 'id': 'lc_run--019c7349-f354-7913-8ca4-811bfc8e4af8-0', 'tool_calls': [{'name': 'multiply', 'args': {'a': 3, 'b': 2}, 'id': 'call_slomVd34Dg7kiYTAgZo3c5iB', 'type': 'tool_call'}], 'invalid_tool_calls': [], 'usage_metadata': 

## Testing with Cloud

We can deploy to Cloud via LangSmith, as outlined [here](https://docs.langchain.com/langsmith/deployment-quickstart#deploy-from-github-with-langgraph-cloud). 

### Create a New Repository on GitHub

* Go to your GitHub account
* Click on the "+" icon in the upper-right corner and select `"New repository"`
* Name your repository (e.g., `langchain-academy`)

### Add Your GitHub Repository as a Remote

* Go back to your terminal where you cloned `langchain-academy` at the start of this course
* Add your newly created GitHub repository as a remote

```
git remote add origin https://github.com/your-username/your-repo-name.git
```
* Push to it
```
git push -u origin main
```

### Connect LangSmith to your GitHub Repository

* Go to [LangSmith](hhttps://smith.langchain.com/)
* Click on `deployments` tab on the left LangSmith panel
* Add `+ New Deployment`
* Then, select the Github repository (e.g., `langchain-academy`) that you just created for the course
* Point the `LangGraph API config file` at one of the `studio` directories
* For example, for module-1 use: `module-1/studio/langgraph.json`
* Set your API keys (e.g., you can just copy from your `module-1/studio/.env` file)

![Screenshot 2024-09-03 at 11.35.12 AM.png](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66dbad4fd61c93d48e5d0f47_deployment2.png)

### Work with your deployment

We can then interact with our deployment a few different ways:

* With the SDK, as before.
* With [LangGraph Studio](https://docs.langchain.com/langsmith/deployment-quickstart#3-test-your-application-in-studio).

![Screenshot 2024-08-23 at 10.59.36 AM.png](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66dbad4fa159a09a51d601de_deployment3.png)

To use the SDK here in the notebook, simply ensure that `LANGSMITH_API_KEY` is set!

In [1]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("LANGSMITH_API_KEY")

In [ ]:
# Replace this with the URL of your deployed graph
URL = "https://langchain-academy-8011c561878d50b1883f7ed11b32d720.default.us.langgraph.app"
client = get_client(url=URL)

# Search all hosted graphs
assistants = await client.assistants.search()

In [37]:
# Select the agent
agent = assistants[0]

In [38]:
agent

{'assistant_id': 'fe096781-5601-53d2-b2f6-0d3403f7e9ca',
 'graph_id': 'agent',
 'created_at': '2024-08-23T17:58:02.722920+00:00',
 'updated_at': '2024-08-23T17:58:02.722920+00:00',
 'config': {},
 'metadata': {'created_by': 'system'}}

In [40]:
from langchain_core.messages import HumanMessage

# We create a thread for tracking the state of our run
thread = await client.threads.create()

# Input
input = {"messages": [HumanMessage(content="Multiply 3 by 2.")]}

# Stream
async for chunk in client.runs.stream(
        thread['thread_id'],
        "agent",
        input=input,
        stream_mode="values",
    ):
    if chunk.data and chunk.event != "metadata":
        print(chunk.data['messages'][-1])

{'content': 'Multiply 3 by 2.', 'additional_kwargs': {'example': False, 'additional_kwargs': {}, 'response_metadata': {}}, 'response_metadata': {}, 'type': 'human', 'name': None, 'id': '8ea04559-f7d4-4c82-89d9-c60fb0502f21', 'example': False}
{'content': '', 'additional_kwargs': {'tool_calls': [{'index': 0, 'id': 'call_EQoolxFaaSVU8HrTnCmffLk7', 'function': {'arguments': '{"a":3,"b":2}', 'name': 'multiply'}, 'type': 'function'}]}, 'response_metadata': {'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_3aa7262c27'}, 'type': 'ai', 'name': None, 'id': 'run-b0ea5ddd-e9ba-4242-bb8c-80eb52466c76', 'example': False, 'tool_calls': [{'name': 'multiply', 'args': {'a': 3, 'b': 2}, 'id': 'call_EQoolxFaaSVU8HrTnCmffLk7', 'type': 'tool_call'}], 'invalid_tool_calls': [], 'usage_metadata': None}
{'content': '6', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'tool', 'name': 'multiply', 'id': '1bf558e7-79ef-4f21-bb66-acafbd04677a', 'tool_call_id': 'c